<center><h1><u> COMMENT ANALYSIS </u></h1></center>
<center><h3><u> USING CLASSIFICATION  </u></h3></center>
<center><h1><u>  </u></h1></center>

In [109]:
import pandas as pd
import numpy as np
import sklearn
import re
from sklearn.feature_extraction.text import CountVectorizer

# LEMMATIZATION 
from nltk.stem import WordNetLemmatizer
from textblob import Word  #datapreparation
# STOP WORDS 
from nltk.corpus import stopwords #datapreparation

import nltk
from nltk.tokenize import word_tokenize
from collections import Counter


#Naive Bayes with count vectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics

#SVM 
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_val_score

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

In [110]:
# PARSING DATA
train = pd.read_csv('data/train.csv',low_memory=False)
labels = pd.read_csv('data/impermium_verification_labels.csv',low_memory=False)
sets = pd.read_csv('data/impermium_verification_set.csv',low_memory=False)



In [111]:
train 

,Insult,Date,Comment
0,1,20120618192155Z,"""You fuck your dad."""
1,0,20120528192215Z,"""i really don't understand your point.\xa0 It ..."
2,0,NaN,"""A\\xc2\\xa0majority of Canadians can and has ..."
3,0,NaN,"""listen if you dont wanna get married to a man..."
4,0,20120619094753Z,"""C\xe1c b\u1ea1n xu\u1ed1ng \u0111\u01b0\u1edd..."
...,...,...,...
3942,1,20120502172717Z,"""you are both morons and that is never happening"""
3943,0,20120528164814Z,"""Many toolbars include spell check, like Yahoo..."
3944,0,20120620142813Z,"""@LambeauOrWrigley\xa0\xa0@K.Moss\xa0\nSioux F..."
3945,0,20120528205648Z,"""How about Felix? He is sure turning into one ..."


In [112]:
labels

,id,Insult,Date,Comment,Usage
0,1,0,20120603163526Z,"""like this if you are a tribe fan""",PrivateTest
1,2,1,20120531215447Z,"""you're idiot.......................""",PrivateTest
2,3,1,20120823164228Z,"""I am a woman Babs, and the only ""war on women...",PrivateTest
3,4,1,20120826010752Z,"""WOW & YOU BENEFITTED SO MANY WINS THIS YEAR F...",PrivateTest
4,5,1,20120602223825Z,"""haha green me red you now loser whos winning ...",PrivateTest
...,...,...,...,...,...
2230,2231,0,20120528100303Z,"""FUCKIN LAME COME ON WTF STOP FUCKING OVER MY ...",PrivateTest
2231,2232,1,20120531185813Z,"""YOU SHUT YOUR IGNORANT PIE HOLE YOU LITTLE IN...",PrivateTest
2232,2233,0,20120529130822Z,"""sweetie pie is looking very much like her cou...",PrivateTest
2233,2234,1,20120531045826Z,"""ball4real where are you with your miami g-ayn...",PrivateTest


In [113]:
train.dropna(inplace = True) # remove null rows 
train.reset_index(inplace = True)
labels.dropna(inplace = True)
labels.reset_index(inplace = True)


In [114]:
#PREPROCESSING DOCUMENTS

In [115]:
# LOWERCASE LETTERS 
train['Comment'] = train['Comment'].apply(lambda x: x.lower())
# REMOVE ENDLINES
train['Comment'] = train['Comment'].str.replace('\n','')
# REMOVE LINKS
train['Comment'] = train['Comment'].str.replace('https?:\/\/.*[\r\n]*',' ')
train['Comment'] = train['Comment'].str.replace('http?:\/\/.*[\r\n]*',' ')

# REMOVE SPECIFIED NON ASCII
train['Comment'] = train['Comment'].str.replace('\u0111' ,' ')

# REMOVE ALL NON ASCII
train['Comment'] = train['Comment'].str.encode("ascii", "ignore")
train['Comment'] = train['Comment'].str.decode("unicode_escape")



### CLASSIFICATION WITH NAIVE BAYES

In [116]:
x_train = train['Comment']
x_test = labels['Comment']
y_train = train['Insult']
y_test = labels['Insult']

count_vectorizer = CountVectorizer()

x_train_cv = count_vectorizer.fit_transform(x_train)
x_test_cv =  count_vectorizer.transform(x_test)

model = GaussianNB()

temp_cv = x_train_cv.toarray()

model.fit(temp_cv ,y_train)
predictions = model.predict(x_test_cv.toarray())
print("Accuracy:",metrics.accuracy_score(y_test, predictions))


Accuracy: 0.5261744966442953


### NAIVE BAYES IMPROVEMENT

In [117]:
stop = stopwords.words('english')
train['Comment'] = train['Comment'].apply(lambda x:" ".join([word for word in x.split() if word not in (stop)])) 
                                                                                    # removing the stopwords
                                                                                    
train['Comment'] = train['Comment'].mask(train['Comment'].map(len) == 1, ' ') # Here we are removing the words with lengh =1 


train['Comment'] = train['Comment'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()])) #Lemmatizing

x_train = train['Comment']
x_test = labels['Comment']
y_train = train['Insult']
y_test = labels['Insult']

count_vectorizer = CountVectorizer(max_df=1.0, min_df=1, ngram_range = (1,2) , max_features=1000)

x_train_cv = count_vectorizer.fit_transform(x_train)
x_test_cv =  count_vectorizer.transform(x_test)


from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()  

temp_cv = x_train_cv.toarray()


#vectorizer
model.fit(temp_cv ,y_train)
predictions = model.predict(x_test_cv.toarray())
print("Accuracy:",metrics.accuracy_score(y_test, predictions))



Accuracy: 0.654586129753915


In [118]:
# A FUNCTION TO COUNT POS TAGS
def count_pos(sentence,pos):
    tokens = word_tokenize(sentence)
    tags = nltk.pos_tag(tokens)

    t = 0
    
    for _,tag in tags:
        if (tag ==pos) :
            t += 1
    if len(tags)==0 :
        return 0
    return ( (t/len(tags)) *100 )      

### ADDING TO THE TRAIN DATA SET 4 EXTRA COLUMNS FOR EACH PART OF SPEECH WITH ITS COUNT

In [119]:
train['NN'] = train['Comment'].apply(lambda x: count_pos(x,'NN'))
train['JJ'] = train['Comment'].apply(lambda x: count_pos(x,'JJ'))
train['VB'] = train['Comment'].apply(lambda x: count_pos(x,'VB'))
train['RB'] = train['Comment'].apply(lambda x: count_pos(x,'RB'))



### PREVIEW OF THE FINAL NEW DF

In [120]:
train

,index,Insult,Date,Comment,NN,JJ,VB,RB
0,0,1,20120618192155Z,"""you fuck dad.""",0.000000,16.666667,0.000000,0.000000
1,1,0,20120528192215Z,"""i really understand point. seems mixing apple...",16.666667,8.333333,8.333333,8.333333
2,4,0,20120619094753Z,"""các bạn xuống đường biểu tình 2011 có ôn hoà ...",57.894737,5.263158,1.052632,0.000000
3,5,0,20120620171226Z,"""@sdl ok, would hope they'd sign one-year cont...",22.580645,19.354839,12.903226,3.225806
4,6,0,20120503012628Z,"""yeah now?""",20.000000,0.000000,0.000000,20.000000
...,...,...,...,...,...,...,...,...
3224,3942,1,20120502172717Z,"""you moron never happening""",0.000000,0.000000,0.000000,16.666667
3225,3943,0,20120528164814Z,"""many toolbars include spell check, like yahoo...",21.739130,8.695652,13.043478,4.347826
3226,3944,0,20120620142813Z,"""@lambeauorwrigley @k.moss sioux falls, s.d. t...",40.000000,4.000000,0.000000,0.000000
3227,3945,0,20120528205648Z,"""how felix? sure turning one hell starting pit...",32.000000,4.000000,8.000000,0.000000


### TF - IDF FOR PART OF SPEECH 


In [121]:
tfidf = TfidfVectorizer()


x_train_tf = tfidf.fit_transform(x_train)
x_test_tf = tfidf.transform(x_test)
 
x_train_tf = pd.DataFrame(x_train_tf.toarray())
x_test_tf = pd.DataFrame(x_test_tf.toarray())

x_train_fin = pd.concat([train['RB'], train['VB'],train['JJ'],train['NN'], x_train_tf], axis=1)
x_test_fin =pd.concat([train['RB'], train['VB'],train['JJ'],train['NN'], x_test_tf], axis=1)

x_train_fin.dropna(inplace = True) # remove null rows 
x_train_fin.reset_index(inplace = True)

x_test_fin.dropna(inplace = True) # remove null rows 
x_test_fin.reset_index(inplace = True)

### SVM WITH TF IDF FOR PART OF SPEECH


In [122]:
model = sklearn.svm.SVC()

#vectorizer
model.fit(x_train_fin,y_train)
predictions = model.predict(x_test_fin)

print("Accuracy:",metrics.accuracy_score(y_test, predictions))

f1s = cross_val_score(model,x_train_fin, y_train, cv=10, scoring='f1_micro')
print ('F1', np.mean(f1s), f1s)

Accuracy: 0.5181208053691275
F1 0.701455685248928 [0.6996904  0.6996904  0.6996904  0.6996904  0.70278638 0.70278638
 0.70278638 0.70278638 0.70278638 0.70186335]


### RANDOM FOREST WITH TF IDF FOR PART OF SPEECH

In [123]:
#Random Forest with tfidf
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(max_depth=2, random_state=0)

#vectorizer
model.fit(x_train_fin ,y_train)
predictions = model.predict(x_test_fin)

print("Accuracy:",metrics.accuracy_score(y_test, predictions))

f1s = cross_val_score(model,x_train_fin, y_train, cv=10, scoring='f1_micro')
print ('F1', np.mean(f1s), f1s)

Accuracy: 0.5181208053691275
F1 0.701455685248928 [0.6996904  0.6996904  0.6996904  0.6996904  0.70278638 0.70278638
 0.70278638 0.70278638 0.70278638 0.70186335]


### IMPROVING F1 SCORE

In [1]:
# from sklearn.svm import SVC 
# from sklearn.model_selection import GridSearchCV

# tuned_parameters = {'kernel':['linear','rbf'],'gamma':[1e-3,1e-4],'C':[1,10,100,1000]}

# model = GridSearchCV(sklearn.svm.SVC(),tuned_parameters)

# model.fit(x_train_fin,y_train)
# print("Better SVM is : ")
# print(model.score(x_test_fin,y_test))


### Notice: I left the improving block to run for about 4 hours and nothing happened , was still running but i was out of time and I had to send the project. Thats the reason that I marked the whole block as a comment. Keep an eye to the code !

## -----------------------------------------------------------------------------------------------------------

### Sources - References 


Most of the ideas - code that i used are based on Project 1 and Project 2 . <br>
Project 1 : https://github.com/bakalianosm/AirBnB-Data-Exploration <br>
Project 2 : https://github.com/bakalianosm/Data-Mining-Clustering-Classification

Stemming - Lematization :https://www.datacamp.com/community/tutorials/stemming-lemmatization-python?utm_source=adwords_ppc&utm_campaignid=9942305733&utm_adgroupid=100189364546&utm_device=c&utm_keyword=&utm_matchtype=b&utm_network=g&utm_adpostion=&utm_creative=255798340456&utm_targetid=aud-517318241987:dsa-929501846124&utm_loc_interest_ms=&utm_loc_physical_ms=9067678&gclid=CjwKCAjw8pH3BRAXEiwA1pvMseromhRhaTZCIswco13hTLG2yBxTds6MnnD3xVVsszDdGz7URPWZyxoC3igQAvD_BwE
        

Stopwords Removal: https://www.geeksforgeeks.org/removing-stop-words-nltk-python/


Laplace Smoothing: https://towardsdatascience.com/introduction-to-na%C3%AFve-bayes-classifier-fa59e3e24aaf

Part of Speech Tagging: https://www.nltk.org/book/ch05.html

Dataframe Concatenation: https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

Links Removal: https://stackoverflow.com/questions/51994254/removing-url-from-a-column-in-pandas-dataframe

Naive - Bayes Classification https://scikit-learn.org/stable/modules/naive_bayes.html